In [ ]:
!pip install roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="2nxKEUjSzlIMWDl3hWnV")

In [ ]:
print(rf.workspace())

loading Roboflow workspace...
{
  "name": "Capstone",
  "url": "capstone-qhqkt",
  "projects": [
    "capstone-qhqkt/cucumber-znek1",
    "capstone-qhqkt/hard-hat-sample-dn68r",
    "capstone-qhqkt/pepper-ldesi",
    "capstone-qhqkt/pepper-parts",
    "capstone-qhqkt/strawberry-parts-d9fnc",
    "capstone-qhqkt/strawberry-rbo1c",
    "capstone-qhqkt/tomato-3ap6q",
    "capstone-qhqkt/tomato-parts"
  ]
}


In [ ]:
project = rf.project("pepper-ldesi")

In [ ]:
records_b3 = []
for page in project.search_all(
    class_name = "b3_pepper",
    in_dataset = True,
    fields = ["name"],
):
    records_b3.extend(page)
print(len(records_b3))

614


In [ ]:
fns_b3 = []
for r in records_b3:
  fns_b3.append(r["name"])
print(len(fns_b3))

614


In [ ]:
project = rf.project("pepper-parts")

In [ ]:
records_pb3 = []
for page in project.search_all(
    class_name = "b3_pepper",
    in_dataset = True,
    fields = ["name"],
):
    records_pb3.extend(page)
print(len(records_pb3))

180


In [ ]:
fns_pb3 = []
for r in records_pb3:
  fns_pb3.append(r["name"])
print(len(fns_pb3))

180


In [ ]:
new_b3s = []
for a in fns_b3:
  if a not in fns_pb3:
    new_b3s.append(a)
print(new_b3s)

In [ ]:
print(len(new_b3s))

434


In [ ]:
import random
random.shuffle(new_b3s)

In [ ]:
new_b3s = new_b3s[:200]

In [ ]:
len(new_b3s)

200

In [ ]:
new_b3s_train = new_b3s[:160]
new_b3s_valid = new_b3s[160:]

In [ ]:
categories_list = [
    'b3_pepper'
]

In [ ]:
import os
import json
part_file_name= '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/new-pepper-part-b3.json'
label_path = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/생리장해'
IBM = {
    "version": "1.0",
    "type": "localization",
    "labels": categories_list,
    "annotations":{}
}
idx= 0
for img in new_b3s:
  json_fn = img.replace('jpg','').replace('JPG','')
  json_fn += 'json'
  json_file_path = os.path.join(label_path,json_fn)
  with open(json_file_path,"r") as json_file:
    data = json.load(json_file)
    image_filename = data["description"]["image"]
    class_name = f"{data['annotations']['disease']}_pepper"
    parts = data["annotations"]["part"]
    value = []
    for p in parts:
      temp = {}
      temp["x"] = int(p["x"]*0.5)
      temp["y"] = int(p["y"]*0.5)
      temp["x2"] = int((p["x"]+p["w"])*0.5)
      temp["y2"] = int((p["y"]+p["h"])*0.5)
      temp["id"] = str(idx)
      idx += 1
      temp["label"] = class_name
      value.append(temp)
    IBM["annotations"][img] = value
with open(part_file_name,"w") as output_json_file:
  json.dump(IBM,output_json_file, ensure_ascii=False)

In [ ]:
project = rf.project("pepper-parts")

In [ ]:
dir_name_i = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/원천데이터/IMG_고추/고추_생리장해/resized_생리장해'
dir_name_a = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/new-pepper-part-b3.json'
for img in new_b3s_train:
  img_path = os.path.join(dir_name_i,img)
  project.upload(img_path,dir_name_a,split='train')
for img in new_b3s_valid:
  img_path = os.path.join(dir_name_i,img)
  project.upload(img_path,dir_name_a,split='valid')